In [19]:
# 필요한 라이브러리 임포트
import os
import json
import cv2
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from PIL import Image
from PIL.ExifTags import TAGS

# 새로운 이미지 크기
new_size = (270, 480)

# 데이터 전처리 함수
def preprocess_data(json_files, img_directory):  
    images = []
    labels = []
    for json_file in json_files:
        with open(json_file, 'r') as f:
            data = json.load(f)
        
        img_file = data['imagePath']  
        img_path = os.path.join(img_directory, img_file)  
        
        # PIL 라이브러리를 이용해 이미지 불러오기
        img = Image.open(img_path)

        # EXIF 데이터로부터 회전 정보 확인 및 이미지 회전
        exif_data = img._getexif()
        if exif_data is not None:
            for tag, value in exif_data.items():
                tag_name = TAGS.get(tag, tag)
                if tag_name == 'Orientation':
                    if value == 3: # 180도 회전
                        img = img.rotate(180)
                    elif value == 6: # 시계방향 270도 회전
                        img = img.rotate(-90)
                    elif value == 8: # 시계방향 90도 회전
                        img = img.rotate(90)

        # numpy 배열로 변환
        img = np.array(img)

        # 이미지 크기 조정
        img = cv2.resize(img, new_size)

        for shape in data['shapes']:
            if shape['label'] == 'tom_flower_poly':
                points = np.array(shape['points'], np.int32)
                cv2.fillPoly(img, [points], (255, 255, 255))

        images.append(img)
        labels.append(1 if len([shape for shape in data['shapes'] if shape['label'] == 'tom_flower_poly']) >= 3 else 0)
    
    images = np.array(images)
    labels = np.array(labels)
    return images, labels

unique, counts = np.unique(labels, return_counts=True)
label_counts = dict(zip(unique, counts))

for label, count in label_counts.items():
    print(f"Label {label}: {count}개")


Label 0: 197개
Label 1: 3개


In [ ]:
def create_model():
    model = Sequential()
    # 입력 이미지의 크기를 조정된 크기로 변경
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(new_size[1], new_size[0], 3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])
    return model

directory = 'E:/tomato_label/097.지능형_스마트팜_통합_데이터(토마토)/01.데이터/1.Training/라벨링데이터/d1.꽃(개화군)'  
json_files = [os.path.join(directory, file) for file in os.listdir(directory) if file.endswith('.json')]
json_files = json_files[:500]

img_directory = 'E:/tomato_label/097.지능형_스마트팜_통합_데이터(토마토)/01.데이터/1.Training/원천데이터/d1.꽃(개화군)/'  

images, labels = preprocess_data(json_files, img_directory)

train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.2, random_state=42)

model = create_model()

model.fit(train_images, train_labels, epochs=10, batch_size=32, validation_data=(test_images, test_labels))

In [28]:
# 필요한 라이브러리 임포트
import os
import json
from PIL import Image, ExifTags
import cv2
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

# 새로운 이미지 크기
new_size = (270, 480)

# 데이터 전처리 함수
def preprocess_data(json_files, img_directory):  
    images = []
    labels = []
    
    for json_file in json_files:
        with open(json_file, 'r') as f:
            data = json.load(f)
        
        img_file = data['imagePath']  
        img_path = os.path.join(img_directory, img_file)  
        
        # PIL 라이브러리를 이용해 이미지 불러오기
        img = Image.open(img_path)

        # EXIF 데이터로부터 회전 정보 확인 및 이미지 회전
        exif_data = img._getexif()
        if exif_data is not None:
            for tag, value in exif_data.items():
                tag_name = ExifTags.TAGS.get(tag, tag)
                if tag_name == 'Orientation':
                    if value == 3: # 180도 회전
                        img = img.rotate(180)
                    elif value == 6: # 시계방향 270도 회전
                        img = img.rotate(-90)
                    elif value == 8: # 시계방향 90도 회전
                        img = img.rotate(90)

        # numpy 배열로 변환
        img = np.array(img)

        # 이미지 크기 조정
        img = cv2.resize(img, new_size)

        for shape in data['shapes']:
            if shape['label'] == 'tom_flower_poly':
                points = np.array(shape['points'], np.int32)
                cv2.fillPoly(img, [points], (255, 255, 255))

        images.append(img)
        labels.append(1 if len([shape for shape in data['shapes'] if shape['label'] == 'tom_flower_poly']) >= 3 else 0)
    
    images = np.array(images)
    labels = np.array(labels)
    return images, labels

# 모델 생성 함수
def create_model():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(new_size[1], new_size[0], 3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])
    return model



# 데이터 디렉토리
directory = 'E:/tomato_label/097.지능형_스마트팜_통합_데이터(토마토)/01.데이터/1.Training/라벨링데이터/d1.꽃(개화군)'  
json_files = [os.path.join(directory, file) for file in os.listdir(directory) if file.endswith('.json')]
json_files = json_files[:500]

img_directory = 'E:/tomato_label/097.지능형_스마트팜_통합_데이터(토마토)/01.데이터/1.Training/원천데이터/d1.꽃(개화군)/'  

# 데이터 전처리
images, labels = preprocess_data(json_files, img_directory)

# 데이터 분할
train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.2, random_state=42)

# 모델 생성
model = create_model()

# 모델 학습
model.fit(train_images, train_labels, epochs=10, batch_size=32, validation_data=(test_images, test_labels))

# 라벨별 데이터 개수 출력
unique, counts = np.unique(labels, return_counts=True)
label_counts = dict(zip(unique, counts))

for label, count in label_counts.items():
    print(f"Label {label}: {count}개")



Epoch 1/10
13/13 [==============================] - 18s 1s/step - loss: 144.4771 - accuracy: 0.9175 - val_loss: 138.0934 - val_accuracy: 0.9900
Epoch 2/10
13/13 [==============================] - 17s 1s/step - loss: 26.8353 - accuracy: 0.9925 - val_loss: 7.0938 - val_accuracy: 0.9900
Epoch 3/10
13/13 [==============================] - 17s 1s/step - loss: 0.9692 - accuracy: 0.9925 - val_loss: 2.1189 - val_accuracy: 0.8300
Epoch 4/10
13/13 [==============================] - 18s 1s/step - loss: 0.2533 - accuracy: 0.9875 - val_loss: 1.5951 - val_accuracy: 0.9800
Epoch 5/10
13/13 [==============================] - 17s 1s/step - loss: 4.8810 - accuracy: 0.9925 - val_loss: 3.2481 - val_accuracy: 0.9900
Epoch 6/10
13/13 [==============================] - 17s 1s/step - loss: 0.2815 - accuracy: 0.9975 - val_loss: 1.3438 - val_accuracy: 0.9800
Epoch 7/10
13/13 [==============================] - 18s 1s/step - loss: 5.3151e-13 - accuracy: 1.0000 - val_loss: 1.9166 - val_accuracy: 0.9800
Epoch 8/10


In [29]:
# 테스트 데이터 전처리 함수
def preprocess_test_images(directory, new_size):
    images = []
    labels = []
    
    # 서브 디렉토리 내의 각 이미지에 대해
    for root, dirs, files in os.walk(directory):
        for img_file in files:
            img_path = os.path.join(root, img_file)
            
            # 이미지 불러오기 및 전처리
            img = Image.open(img_path)
            img = np.array(img)
            img = cv2.resize(img, new_size)
            
            images.append(img)
            labels.append(1 if "blossomed_3" in img_file else 0)
    
    return np.array(images), np.array(labels)

In [30]:
# 테스트 데이터 전처리
blossomed_3_directory = "E:\\tomato_data\\TOMATO_BLOOMING_IMAGE_ENV_20231124\\TOMATO_BLOOMING_IMAGE_ENV_20231124\\blossomed_3"
blossomed_1_directory = "E:\\tomato_data\\TOMATO_BLOOMING_IMAGE_ENV_20231124\\TOMATO_BLOOMING_IMAGE_ENV_20231124\\blossomed_1"

test_images_3, test_labels_3 = preprocess_test_images(blossomed_3_directory, new_size)
test_images_1, test_labels_1 = preprocess_test_images(blossomed_1_directory, new_size)

# 모델 예측
predictions_3 = model.predict(test_images_3)
predictions_1 = model.predict(test_images_1)



25/25 [==============================] - 5s 204ms/step


In [24]:
# 예측 결과를 이진 값으로 변환
predictions_3_binary = (predictions_3 > 0.5).astype("int32")
predictions_1_binary = (predictions_1 > 0.5).astype("int32")

# 예측 결과 출력
print("Predictions for 'blossomed_3' images:")
for i, prediction in enumerate(predictions_3_binary):
    print(f"Image {i+1}: {'Blossomed' if prediction == 1 else 'Not blossomed'}")

print("\nPredictions for 'blossomed_1' images:")
for i, prediction in enumerate(predictions_1_binary):
    print(f"Image {i+1}: {'Blossomed' if prediction == 1 else 'Not blossomed'}")


Predictions for 'blossomed_3' images:
Image 1: Not blossomed
Image 2: Not blossomed
Image 3: Not blossomed
Image 4: Not blossomed
Image 5: Not blossomed
Image 6: Not blossomed
Image 7: Not blossomed
Image 8: Not blossomed
Image 9: Blossomed
Image 10: Not blossomed
Image 11: Not blossomed
Image 12: Blossomed
Image 13: Blossomed
Image 14: Not blossomed
Image 15: Not blossomed
Image 16: Not blossomed
Image 17: Not blossomed
Image 18: Not blossomed
Image 19: Not blossomed
Image 20: Blossomed
Image 21: Blossomed
Image 22: Not blossomed
Image 23: Not blossomed
Image 24: Blossomed
Image 25: Blossomed
Image 26: Not blossomed
Image 27: Not blossomed
Image 28: Not blossomed
Image 29: Blossomed
Image 30: Blossomed
Image 31: Not blossomed
Image 32: Blossomed
Image 33: Not blossomed
Image 34: Blossomed
Image 35: Not blossomed
Image 36: Not blossomed
Image 37: Blossomed
Image 38: Not blossomed
Image 39: Blossomed
Image 40: Not blossomed
Image 41: Blossomed
Image 42: Not blossomed
Image 43: Not bloss

In [32]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score

# Confusion matrix 
cm_3 = confusion_matrix(test_labels_3, predictions_3_binary)
cm_1 = confusion_matrix(test_labels_1, predictions_1_binary)

print("Confusion matrix for 'blossomed_3' images:")
print(cm_3)

print("\nConfusion matrix for 'blossomed_1' images:")
print(cm_1)

# Accuracy
accuracy_3 = accuracy_score(test_labels_3, predictions_3_binary)
accuracy_1 = accuracy_score(test_labels_1, predictions_1_binary)

print(f"\nAccuracy for 'blossomed_3' images: {accuracy_3*100:.2f}%")
print(f"Accuracy for 'blossomed_1' images: {accuracy_1*100:.2f}%")

# Precision
precision_3 = precision_score(test_labels_3, predictions_3_binary)
precision_1 = precision_score(test_labels_1, predictions_1_binary)

print(f"\nPrecision for 'blossomed_3' images: {precision_3*100:.2f}%")
print(f"Precision for 'blossomed_1' images: {precision_1*100:.2f}%")

# Recall
recall_3 = recall_score(test_labels_3, predictions_3_binary)
recall_1 = recall_score(test_labels_1, predictions_1_binary)

print(f"\nRecall for 'blossomed_3' images: {recall_3*100:.2f}%")
print(f"Recall for 'blossomed_1' images: {recall_1*100:.2f}%")


Confusion matrix for 'blossomed_3' images:
[[365 126]
 [  0   0]]

Confusion matrix for 'blossomed_1' images:
[[678  91]
 [  0   0]]

Accuracy for 'blossomed_3' images: 74.34%
Accuracy for 'blossomed_1' images: 88.17%

Precision for 'blossomed_3' images: 0.00%
Precision for 'blossomed_1' images: 0.00%

Recall for 'blossomed_3' images: 0.00%
Recall for 'blossomed_1' images: 0.00%


D:\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
